#### Bonus : Implémenter votre propre algorithme KNN
Nous allons essayer de programmer une KNN sur le data set réduit avec les 23 variables au lieu de 32.

In [80]:

import pandas as pd

import numpy as np
from time import time

In [123]:
data1= pd.read_csv('kNN.csv')

In [124]:
#la colonne Unnamed est completement vide, nous allons l'enlever du data set
data1 = data1.drop(['id',"Unnamed: 32"], axis=1)

data1['diagnosis']= data1['diagnosis'].astype('category').cat.codes
print("Encodage de M et B par ", data1['diagnosis'].unique())

Encodage de M et B par  [1 0]


In [83]:
# définition des labels 'y' et X les features
#Suppression des features qui possèdent une correlation
# infeieur à 0.35 avec la target diaganostic.
data = data1.drop(["symmetry_mean","fractal_dimension_mean","texture_se","perimeter_se","smoothness_se","symmetry_se","fractal_dimension_se","fractal_dimension_worst"], axis=1)

y = data['diagnosis']

X = data.drop(["diagnosis"], axis=1)



In [84]:
X.shape # une observation d'un patient est décrite à l'aide de 22 features.

(569, 22)

In [85]:
# calcuer la distance euclidienne entre deux observation (patients)
def distance (a,b):
    dis = ((a - b)**2).sum() ** 0.5
    return round(dis,4)

# calcuer la moyenne de la liste des distances
def disMean (listDis):
    # dans la liste figure toujours la distance d'une observation avec elle même d'ou la 
    # len(lisDis)-1
    result = sum(listDis) /(len(listDis)-1)
    return result

# imprime le dictionnaire
def printDic(dic):
    for k in dic.keys():
        print(k , ":", dic[k])

# Standard scaler Manuel
def StandScal (X) :
    X_ss = (X - X.mean(axis = 0))/ X.std(axis=0)
    return X_ss      

In [128]:
# alogorithme principal

m = 10
# Standardisation 
X_ss = StandScal(X)

# faire un test avec un dataset de taille m
data= X_ss.iloc[:m,:]


# Initialiser k au nombre de plus proches voisins choisi
k= 3

# structure de données qui recoit les résultats sous forme de dic ou Keys : index obersvation
# values représentent les  distance
# dictance_e = {j : sort}
# sort est la liste des index des k distance les plus faible renvoyé par argsort() 
# de la liste des distance  dic (préalablement transformée sour forme d'un array)



dictance_e = {}

# demarrage à partir de la ligne j = 0
j = 0
# un dictionnaire qui contient en keys le tuple(i,j) , values : la distance entre le tupe (i, j)
counted ={(0,0):0}

for j in range(data.shape[0]):
    # dis : la liste qui contien
    dis = [] 
    
    a = data.iloc[j,:]
    
    for i in range(data.shape[0]):
        if (i,j) not in counted.keys() or (j,i) not in counted.keys():
            b = data.iloc[i,:]
            d = distance (a,b)  
            counted[(i,j)] = d 
        else :
            d = counted[(i,j)]
        
        dis.append(d)
        
           
# Trier cette collection ordonnée contenant distances et indices de la plus petite distance à la plus grande (dans ordre croissant).
#list to array
    adis =np.array(dis)
    sort = adis.argsort()
    liste=np.delete(sort,0) # le premier element correspondant à la distance 0 avec soi-meme
    
#Obtenir les étiquettes des k entrées sélectionnées  
# k +1 car la distance d'une observation avec elle même est égale à 0
    dictance_e[j] = liste[:k]
    


In [129]:
# i : une obervation i pour laquel on veut faire une prédication

i = 4
# diag est la 
List_distance = []
print(list(dictance_e.get(i)))
for i in list(dictance_e.get(i)):
       List_distance.append(y[i])
        
value = np.mean(List_distance)

print("label true :", y[i])
print("label predict :", value)
      
if value > 0.5 :
     print('diag = M')
elif value < 0.5 :
     print ('diag = B')
else :
     print("% M  % B = % 50")

[1, 2, 6]
label true : 1
label predict : 1.0
diag = M


In [130]:
#printDic(dictance_e)

In [91]:
# optimissation du code précedent
def KNN_distance(X,m,k):
    # m : dimension de test 
    # X : features
    # k : knn 
    
    #m = 100
     # Initialiser k au nombre de plus proches voisins choisi
    #k= 3

    # Standardisation 
    X_ss = StandScal(X)

    # faire un test avec un dataset de taille m
    data= X_ss.iloc[:m,:]


   
    # structure de données qui recoit les résultats sous forme de dic ou Keys : index obersvation
    # values représentent les  distance
    # dictance_e = {j : sort}
    # sort est la liste des index des k distance les plus faible renvoyé par argsort() 
    # de la liste des distance  dic (préalablement transformée sour forme d'un array)


    dictance_e = {}

    # demarrage à partir de la ligne j = 0
    j = 0
    # un dictionnaire qui contient en keys le tuple(i,j) , values : la distance entre le tupe (i, j)
    counted ={(0,0):0}

    for j in range(data.shape[0]):
        # dis : la liste qui contien
        dis = [] 
    
        a = data.iloc[j,:]
    
        for i in range(data.shape[0]):
            if (i,j) not in counted.keys() or (j,i) not in counted.keys():
                b = data.iloc[i,:]
                d = distance (a,b)  
                counted[(i,j)] = d 
            else :
                d = counted[(i,j)]
        
            dis.append(d)
        
           
    # Trier cette collection ordonnée contenant distances et indices de la plus petite distance à la plus grande (dans ordre croissant).
    #list to array
         
        adis =np.array(dis)
        sort = adis.argsort()
        liste=np.delete(sort,0)
    
    #Obtenir les étiquettes des k entrées sélectionnées  
    # k +1 car la distance d'une observation avec elle même est égale à 0
    
        dictance_e[j] = liste[:k]
    
    return dictance_e
    


In [125]:
def KNN (k , i , dictance_e):
    
    List_distance = []
    
    for i in list(dictance_e.get(i)):
        List_distance.append(y[i])
        
    value = np.mean(List_distance)

    print("label true :", y[i])
    print("label predict :", value)
      
    if value > 0.5 :
            print('diag = M')
    elif value < 0.5 :
        print ('diag = B')
    else :
        print("% M  % B = % 50")
    

In [111]:
def predict (X, m, k , i):
    # X : features
    # m : dimension de test 
    # k : knn 
    # i : element à prédire
    dic_distance = KNN_distance(X,m,k)
    #printDic (dic_distance)
    KNN (k , i , dic_distance)


In [126]:
# dic_distance
predict(X,50,3,5)


label true : 1
label predict : 1.0
diag = M
